<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/303_HITL_FastAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FastAPI server for HITL Orchestrator API

In [ ]:
"""FastAPI server for HITL Orchestrator API"""

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
from typing import Optional, Dict, Any
from datetime import datetime
from agents.hitl_orchestrator.api import process_task_request, get_task_status

# Create FastAPI app
app = FastAPI(
    title="HITL Orchestrator API",
    description="Human-in-the-Loop Collaboration Orchestrator API for routing AI agent outputs",
    version="1.0.0"
)


# Request/Response Models
class TaskRequest(BaseModel):
    """Task request model"""
    task_id: str = Field(..., description="Unique task identifier")
    task_type: str = Field(..., description="Type of task (e.g., 'document_classification')")
    task_description: str = Field(..., description="Description of the task")
    domain: str = Field(..., description="Domain (e.g., 'customer_support')")
    risk_level: str = Field(..., description="Risk level: 'low', 'medium', or 'high'")
    timestamp: Optional[str] = Field(None, description="ISO timestamp (defaults to now)")


class AgentOutputRequest(BaseModel):
    """Agent output request model"""
    predicted_label: Optional[str] = Field(None, description="Predicted label/classification")
    explanation: Optional[str] = Field(None, description="Explanation of the prediction")
    # Allow additional fields
    class Config:
        extra = "allow"


class ProcessTaskRequest(BaseModel):
    """Complete task processing request"""
    task: TaskRequest
    agent_output: AgentOutputRequest
    confidence_score: float = Field(..., ge=0.0, le=1.0, description="Confidence score (0.0-1.0)")
    human_review: Optional[Dict[str, Any]] = Field(None, description="Optional existing human review")


class RoutingDecisionResponse(BaseModel):
    """Routing decision response"""
    task_id: str
    routing_decision: Dict[str, Any]
    final_decision: Dict[str, Any]
    audit_log: Dict[str, Any]
    status: str
    agent_output: Dict[str, Any]
    confidence_score: float
    risk_level: str


class HealthResponse(BaseModel):
    """Health check response"""
    status: str
    service: str
    version: str


# API Endpoints
@app.get("/", response_model=HealthResponse)
async def root():
    """Root endpoint - health check"""
    return {
        "status": "healthy",
        "service": "HITL Orchestrator API",
        "version": "1.0.0"
    }


@app.get("/health", response_model=HealthResponse)
async def health_check():
    """Health check endpoint"""
    return {
        "status": "healthy",
        "service": "HITL Orchestrator API",
        "version": "1.0.0"
    }


@app.post("/api/v1/process-task", response_model=RoutingDecisionResponse)
async def process_task(request: ProcessTaskRequest):
    """
    Process a single task and return routing decision.

    This endpoint accepts a task with agent output and confidence score,
    applies the routing policy, and returns the routing decision.
    """
    try:
        # Convert Pydantic models to dictionaries
        task_dict = request.task.dict()
        agent_output_dict = request.agent_output.dict()

        # Add timestamp if not provided
        if not task_dict.get("timestamp"):
            task_dict["timestamp"] = datetime.now().isoformat()

        # Process the task
        result = process_task_request(
            task=task_dict,
            agent_output=agent_output_dict,
            confidence_score=request.confidence_score,
            human_review=request.human_review
        )

        return RoutingDecisionResponse(**result)

    except ValueError as e:
        raise HTTPException(status_code=400, detail=str(e))
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Internal server error: {str(e)}")


@app.get("/api/v1/task-status/{task_id}")
async def get_task_status_endpoint(task_id: str):
    """
    Get status of a processed task.

    Note: This is a placeholder for MVP. In production, this would query a database.
    """
    try:
        status = get_task_status(task_id)
        return status
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Internal server error: {str(e)}")


# Run server (for development)
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)





# 🧠 Big Picture First (This Is the Key)

You already built:

* an **agent**
* decision logic
* real-time processing functions

But right now, that agent can only be used by:

* Python scripts
* tests
* developers

**FastAPI turns your agent into a live web service.**

It lets:

* other programs
* other teams
* other machines
* frontends
* humans via browsers

talk to your agent over the internet.

---

# 🚪 Mental Model: FastAPI = Reception Desk

Think of your system like a company:

* The **agent** is the expert team in the back
* The **API functions** are internal phone extensions
* **FastAPI** is the front desk

The front desk:

* accepts requests
* checks they make sense
* routes them to the right expert
* formats the response nicely
* handles errors politely

---

# 🧩 What FastAPI Is (Plain English)

FastAPI is a framework that:

* listens for HTTP requests (web calls)
* converts JSON into Python objects
* validates inputs
* calls your logic
* sends back JSON responses

You are **not** rewriting your agent.

You are **wrapping it** so others can use it.

---

# 🧱 `app = FastAPI(...)`

This creates:

> “A web server that knows how to handle requests.”

The metadata (`title`, `description`, `version`) is not fluff:

* it auto-generates docs
* helps other teams understand your service
* makes this feel “official”

---

# 🧾 Pydantic Models (Why So Many Classes?)

These classes define **the contract**.

They answer:

> “What does a valid request look like?”
> “What does a valid response look like?”

### Think of them like forms:

* required fields
* allowed values
* descriptions

This prevents:

* bad data
* confusion
* silent failures

And FastAPI uses them to:

* auto-validate input
* auto-generate documentation
* auto-serialize responses

This is **huge** for trust and usability.

---

# 📥 Request Models (Inputs)

### `TaskRequest`

Defines:

* what a task must contain
* what risk looks like
* what domain means

No task enters your system unless it matches this shape.

---

### `AgentOutputRequest`

Defines:

* what the AI produced
* allows flexibility (`extra = "allow"`)

This future-proofs your API for richer agent outputs.

---

### `ProcessTaskRequest`

Bundles everything together:

* task
* agent output
* confidence
* optional human review

This mirrors your **mental model perfectly**.

---

# 📤 Response Models (Outputs)

### `RoutingDecisionResponse`

This guarantees:

* every caller gets the same structured answer
* no surprises
* no hidden fields

This makes your agent:

* predictable
* integrable
* testable by others

---

# ❤️ Health Endpoints (`/` and `/health`)

These are boring but critical.

They answer:

> “Is the service alive?”

Used by:

* load balancers
* monitoring systems
* DevOps
* Kubernetes

If this endpoint works, the service is up.

---

# 🚦 The Real Endpoint: `/api/v1/process-task`

This is **the money endpoint**.

This is what external systems call.

### What it does (conceptually):

1. Accepts a task request
2. Validates everything
3. Adds a timestamp if missing
4. Calls your **existing agent logic**
5. Returns the result in a clean format

Important insight:

> **FastAPI does not decide anything.**
> Your agent does.

FastAPI just makes the agent reachable.

---

# ⚠️ Error Handling

```python
raise HTTPException(...)
```

This is how your service:

* fails gracefully
* tells callers what they did wrong
* avoids crashing

This is critical once you’re “on the internet”.

---

# ▶️ `uvicorn.run(...)`

This starts the server.

It’s like saying:

> “Start answering the phone.”

Once this runs:

* your API is live
* requests can come in
* your agent is operational

---

# 🎯 Why You Actually Need FastAPI (The Real Answer)

You do **not** need FastAPI to:

* build logic
* test locally
* understand agents

You need FastAPI to:

> **Deploy your agent into the real world.**

Without it:

* no frontend can call you
* no other team can integrate
* no workflow can depend on it

With it:

* your HITL orchestrator becomes shared infrastructure
* governance is centralized
* trust logic is reused everywhere

---

# 🧠 The “Aha” Moment (Most Important)

This FastAPI layer is what turns:

> *“We built a HITL agent”*

into:

> **“We offer HITL decision governance as a service.”**

That’s a massive upgrade.

---

# 🏁 Final Takeaway

If this still feels like “a lot of boilerplate”, that’s good.

It means:

* the hard thinking is already done
* the complexity is about *access*, not *intelligence*

You’ve crossed the line from **agent design** to **platform engineering**.




# Test HTTP API endpoints for HITL Orchestrator

In [ ]:
"""Test HTTP API endpoints for HITL Orchestrator"""

import pytest
from fastapi.testclient import TestClient
from agents.hitl_orchestrator.api_server import app
from datetime import datetime

# Create test client
client = TestClient(app)


def test_health_check():
    """Test health check endpoint"""
    response = client.get("/health")
    assert response.status_code == 200
    data = response.json()
    assert data["status"] == "healthy"
    assert data["service"] == "HITL Orchestrator API"
    print("\n✅ Health check test passed!")


def test_root_endpoint():
    """Test root endpoint"""
    response = client.get("/")
    assert response.status_code == 200
    data = response.json()
    assert data["status"] == "healthy"
    print("✅ Root endpoint test passed!")


def test_process_task_auto_approve():
    """Test task processing with auto-approve scenario"""
    request_data = {
        "task": {
            "task_id": "api_test_001",
            "task_type": "document_classification",
            "task_description": "Classify customer email",
            "domain": "customer_support",
            "risk_level": "low"
        },
        "agent_output": {
            "predicted_label": "Billing Issue",
            "explanation": "Email contains references to charges"
        },
        "confidence_score": 0.92
    }

    response = client.post("/api/v1/process-task", json=request_data)
    assert response.status_code == 200
    data = response.json()

    assert data["status"] == "auto_approved"
    assert data["final_decision"]["final_decision"] == "approved"
    assert data["final_decision"]["decision_source"] == "agent"
    assert data["routing_decision"]["routing_decision"] == "auto_approve"

    print("\n✅ Auto-approve API test passed!")
    print(f"   Task ID: {data['task_id']}")
    print(f"   Status: {data['status']}")


def test_process_task_human_review():
    """Test task processing with human review scenario"""
    request_data = {
        "task": {
            "task_id": "api_test_002",
            "task_type": "document_classification",
            "task_description": "Classify refund request",
            "domain": "customer_support",
            "risk_level": "medium"
        },
        "agent_output": {
            "predicted_label": "Refund Request",
            "explanation": "Customer explicitly asks for a refund"
        },
        "confidence_score": 0.65
    }

    response = client.post("/api/v1/process-task", json=request_data)
    assert response.status_code == 200
    data = response.json()

    assert data["status"] == "pending_review"
    assert data["routing_decision"]["routing_decision"] == "human_review"
    assert data["routing_decision"]["assigned_human_role"] == "domain_reviewer"

    print("\n✅ Human review API test passed!")
    print(f"   Task ID: {data['task_id']}")
    print(f"   Status: {data['status']}")
    print(f"   Assigned Role: {data['routing_decision']['assigned_human_role']}")


def test_process_task_with_review():
    """Test task processing with existing human review"""
    request_data = {
        "task": {
            "task_id": "api_test_003",
            "task_type": "policy_decision",
            "task_description": "Determine eligibility",
            "domain": "account_management",
            "risk_level": "medium"
        },
        "agent_output": {
            "predicted_label": "Not Eligible",
            "explanation": "Account does not meet requirements"
        },
        "confidence_score": 0.70,
        "human_review": {
            "review_id": "review_api_003",
            "task_id": "api_test_003",
            "human_role": "domain_reviewer",
            "human_decision": "approve",
            "human_feedback": "Correctly identified",
            "confidence_assessment": "medium",
            "timestamp": datetime.now().isoformat()
        }
    }

    response = client.post("/api/v1/process-task", json=request_data)
    assert response.status_code == 200
    data = response.json()

    assert data["status"] == "reviewed"
    assert data["final_decision"]["final_decision"] == "approved"
    assert data["final_decision"]["decision_source"] == "human"

    print("\n✅ Existing review API test passed!")
    print(f"   Task ID: {data['task_id']}")
    print(f"   Status: {data['status']}")
    print(f"   Final Decision: {data['final_decision']['final_decision']}")


def test_process_task_escalation():
    """Test task processing with escalation scenario"""
    request_data = {
        "task": {
            "task_id": "api_test_004",
            "task_type": "policy_decision",
            "task_description": "High-risk decision",
            "domain": "account_management",
            "risk_level": "high"
        },
        "agent_output": {
            "predicted_label": "Not Eligible",
            "explanation": "High-risk decision"
        },
        "confidence_score": 0.85
    }

    response = client.post("/api/v1/process-task", json=request_data)
    assert response.status_code == 200
    data = response.json()

    assert data["routing_decision"]["routing_decision"] == "escalate"
    assert data["routing_decision"]["assigned_human_role"] == "senior_manager"

    print("\n✅ Escalation API test passed!")
    print(f"   Task ID: {data['task_id']}")
    print(f"   Routing Decision: {data['routing_decision']['routing_decision']}")
    print(f"   Assigned Role: {data['routing_decision']['assigned_human_role']}")


def test_process_task_validation_error():
    """Test validation error handling"""
    # Missing required field
    request_data = {
        "task": {
            "task_id": "api_test_005",
            # Missing task_type, task_description, etc.
        },
        "agent_output": {},
        "confidence_score": 0.75
    }

    response = client.post("/api/v1/process-task", json=request_data)
    assert response.status_code == 422  # Validation error
    print("\n✅ Validation error test passed!")


def test_process_task_invalid_confidence():
    """Test invalid confidence score handling"""
    request_data = {
        "task": {
            "task_id": "api_test_006",
            "task_type": "document_classification",
            "task_description": "Test",
            "domain": "customer_support",
            "risk_level": "low"
        },
        "agent_output": {
            "predicted_label": "Test"
        },
        "confidence_score": 1.5  # Invalid: > 1.0
    }

    response = client.post("/api/v1/process-task", json=request_data)
    assert response.status_code == 422  # Validation error
    print("✅ Invalid confidence score test passed!")


def test_get_task_status():
    """Test task status endpoint"""
    response = client.get("/api/v1/task-status/test_task_001")
    assert response.status_code == 200
    data = response.json()
    assert "task_id" in data
    assert "status" in data
    print("\n✅ Task status endpoint test passed!")


if __name__ == "__main__":
    print("Testing HITL Orchestrator API Endpoints...")
    print("=" * 60)

    test_health_check()
    test_root_endpoint()
    test_process_task_auto_approve()
    test_process_task_human_review()
    test_process_task_with_review()
    test_process_task_escalation()
    test_process_task_validation_error()
    test_process_task_invalid_confidence()
    test_get_task_status()

    print("\n" + "=" * 60)
    print("✅ All API endpoint tests passed!")

